# 양방향 LSTM과 어텐션 메커니즘(BiLSTM with Attention mechanism)

단뱡항 LSTM으로 텍스트 분류를 수행할 수도 있지만 때로는 양방향 LSTM을 사용하는 것이 더 강력합니다. 여기에 추가적으로 어텐션 메커니즘을 사용할 수도 있습니다. 양방향 LSTM과 어텐션 메커니즘으로 IMDB 리뷰 감성 분류하기를 수행해봅시다.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# 바다나우 어텐션(Bahdanau Attention)

어텐션 스코어 함수란 주어진 query와 모든 key에 대해서 유사도를 측정하는 함수를 말합니다. 그리고 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)이었습니다. 다음은 닷 프로덕트 어텐션의 어텐션 스코어 함수를 보여줍니다.

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


# 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

#케라스의 함수형 API를 사용
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)


양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

In [ ]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태
#양방향이라서 두가지 상태에 다 주어야 한다.

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)
#순방향으로 학습하는 녀석 , Dense없으면 할수 있는것도 없
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
# Attention 후 예측
word_to_index = imdb.get_word_index()
index_to_word = {}

for k, value in word_to_index.items():
  index_to_word [value] = k

import re

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^0-9a-zA-Z ]','', new_sentence).lower()
  encoded = []  # 정수 인코딩
  for word in new_sentence.split():
    try:
      if word_to_index[word] <= 10000:
        encoded.append(word_to_index[word] + 3)   # 0<pad>,1<sos>,2<unk>는 imdb에서 사용
      else:
        encoded.append(2)  # 10000 이상의 값은 <unk> 취급
    except KeyError:
      encoded.append(2)  # 단어 집합에 없는 경우 <unk> 취급

  print('encoded : ', encoded)
  pad_new = pad_sequences([encoded], maxlen=max_len)

  # predict
  score = float(model.predict(pad_new))

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정".format((1 - score) * 100))

print(sentiment_predict("The movie is so much fun, I recommend it."))
print(sentiment_predict("It's boring and dull, and even the screenplay is a mess."))